In [1]:
import os
import pandas as pd
import nltk
import os
import nltk
import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-09 13:04:13.483928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744196653.502495   19298 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744196653.508251   19298 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744196653.523370   19298 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17441

/Lab1


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")

print("Path to dataset files:", path)

#PATH
filepath = '/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/Books_rating.csv'


Path to dataset files: /root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1


In [3]:
data=pd.read_csv(filepath)
print(len(data))
display(data[0:1].T)
data=data[0:100]

3000000


,0
Id,1882931173
Title,Its Only Art If Its Well Hung!
Price,NaN
User_id,AVCGYZL8FQQTD
profileName,"Jim of Oz ""jim-of-oz"""
review/helpfulness,7/7
review/score,4.0
review/time,940636800
review/summary,Nice collection of Julie Strain images
review/text,This is only for Julie Strain fans. It's a col...


In [4]:
inputt=data['review/text']
labelt=data['review/score']

print(len(inputt))
print(labelt[0])

300000
4.0


In [5]:
import re
sentences = []
labels = []
for i, text in enumerate(inputt):
    if not isinstance(text, str):  # Convert non-strings to empty string
        text = str(text) if text is not None else ''
    
    text = text.lower()
    text = re.sub(r'[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', text)  # Remove emails
    text = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', text)  # Remove IPs
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\d+(\.\d+)?', '', text) #Remove numbers
    sentences.append(text)

labels = [int(round(x)) for x in labelt]

print(sentences[1])
print(labels[0])


i dont care much for dr seuss but after reading philip nels book i changed my mindthats a good testimonial to the power of rels writing and thinking rel plays dr seuss the ultimate compliment of treating him as a serious poet as well as one of the th centurys most interesting visual artists and after reading his book i decided that a trip to the mandeville collections of the library at university of california in san diego was in order so i could visit some of the incredible seussgeisel holdings they have theretheres almost too much to take in for like william butler yeats seuss led a career that constantly shifted and metamoprhized itself to meet new historical and political cirsumstances so he seems to have been both a leftist and a conservative at different junctures of his career both in politics and in art as nel shows us he was once a cartoonist for the fabled pm magazine and like andy warhol he served his time slaving in the ad business too all was in the service of amusing and 

In [6]:
from transformers import BertTokenizer

num_labels= 6

# Load the tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',num_labels=num_labels)
print(len(sentences))
# Tokenize the sentences
encoding = tokenizer(
    sentences,                      # List of sentences
    add_special_tokens=True,        # Add [CLS] and [SEP] tokens
    max_length=128,                 # Truncate/pad to this max length
    padding='max_length',           # Pad all sentences to the same length
    truncation=True,                # Truncate sentences longer than max_length
    return_tensors="pt"             # Return PyTorch tensors
)

# Extract input_ids and attention_mask
input_ids = encoding['input_ids']            # Shape: [num_sentences, max_length]
attention_mask = encoding['attention_mask']  # Shape: [num_sentences, max_length]



300000


In [7]:
encoding['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [8]:

labels = torch.tensor(labels)  # Shape: [num_sentences]
print(len(input_ids))
#print(labels[0:2000])

300000


In [9]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

# Create a dataset
dataset = TensorDataset(input_ids, attention_mask, labels)
print(len(dataset))

# Specify dataset sizes (e.g., 70% train, 15% validation, 15% test)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Randomly split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
batch_s = 64
train_loader = DataLoader(train_dataset, batch_size=batch_s, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_s)
test_loader = DataLoader(test_dataset, batch_size=batch_s)

300000


In [10]:
from transformers import BertForSequenceClassification
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizer
from transformers import BertModel
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print(torch.cuda.is_available())  # Returns True if a GPU is available
# Specify the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move your model to the device
model = model.to(device)

# Move your data tensors to the device


True


In [12]:
epochs = 5
learning_rate = 1e-5
optimizer = optim.AdamW(model.parameters(),lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
best_loss = 10000

In [13]:
import wandb
wandb.init(project="Group_19_lab1")
wandb.config.update({
    "learning_rate": learning_rate
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wilwoh-2 (ertveh-4-lule-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:

best_loss = 10000
for epoch in range(epochs):
    avg_train_loss = 0
    avg_val_loss = 0

    model.train()
    for batch in train_loader:
        input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
        attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
        labels = batch[2].to(device)              # Move labels to GPU/CPU
        output = model(input_ids,attention_mask=attention_mask)
        output = output.logits

        loss = loss_fn(output,labels)
        avg_train_loss += loss.item()

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        

    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
            attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
            labels = batch[2].to(device)              # Move labels to GPU/CPU
            output = model(input_ids,attention_mask=attention_mask)
            output = output.logits
            loss = loss_fn(output,labels)
            if loss < best_loss:
                best_loss = loss
                best_model = model
            avg_val_loss += loss.item()
            for i in range(len(output)):
                o = torch.argmax(output[i]).item()
                label = labels[i].item()
                total += 1
                if  o == label:
                    correct += 1
            
    torch.cuda.empty_cache()
    
        
    avg_train_loss = avg_train_loss/len(train_loader)
    avg_val_loss = avg_val_loss/len(val_loader)
    val_acc = correct/total
    wandb.log({
        "train_loss":avg_train_loss,
        "val_loss":avg_val_loss,
        "val_acc":val_acc
    })
    del input_ids, attention_mask, labels, output, loss

    
    print("Epoch: ",epoch,"Train loss: ",avg_train_loss,"Val loss: ", avg_val_loss)

Epoch:  0 Train loss:  0.8083270762585635 Val loss:  0.7436480961994014
Epoch:  1 Train loss:  0.6990283356978354 Val loss:  0.7201673224229704
Epoch:  2 Train loss:  0.632366037446867 Val loss:  0.7433671588924798
Epoch:  3 Train loss:  0.5639021888257235 Val loss:  0.7639821552756157
Epoch:  4 Train loss:  0.49503342294696623 Val loss:  0.7776042968458072


In [15]:

total = 0
correct = 0
best_model.eval()
best_model.to(device)
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
        attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
        labels = batch[2].to(device)              # Move labels to GPU/CPU
        output = best_model(input_ids,attention_mask=attention_mask)
        output = output.logits
        
        for i in range(len(output)):
            o = torch.argmax(output[i]).item()
            label = labels[i].item()
            total += 1
            if  o == label:
                correct += 1
print(correct/total)
    

0.7179333333333333


In [16]:

#torch.save(best_model.state_dict(), "model_name")

print("Model saved!")

Model saved!
